# Data Acquisition

Importing libraries

In [152]:
import sqlalchemy as chemy
import pandas as pd
import numpy as np

Engine

In [153]:
engine = chemy.create_engine("sqlite:///../data/raw/nicolascortinas.db")

Getting all three tables: business_info, personal_info, rank_info

In [164]:
b = pd.read_sql_table("business_info", engine)
p = pd.read_sql_table("personal_info", engine)
r = pd.read_sql_table("rank_info", engine)

Data Wrangling -> business_info table

In [447]:
p = pd.read_sql_table("personal_info", engine)
p_original = p

In [166]:
b_original = b

In [167]:
b

,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
0,7468,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,4605,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,7368,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,3808,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,3213,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5
...,...,...,...,...,...,...,...
2203,7578,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134
2204,9196,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134
2205,1175,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856
2206,9466,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978


In [173]:
null_cols = b.isnull().sum() 
print(null_cols)
print("")
print(null_cols[null_cols > 0] / len(b) * 100)
print(len(b))

# column realTimeWorth of the business_info table is the only column with null values. The column is empty. 

id                     0
Unnamed: 0             0
Source                 0
worth                  0
worthChange            0
realTimeWorth       2208
realTimePosition       0
dtype: int64

realTimeWorth    100.0
dtype: float64
2208


In [174]:
drop_cols = list(null_cols[null_cols > 0].index)
drop_cols

['realTimeWorth']

In [177]:
# additional columns to drop will be worthChange and realTimePosition since dont provide any meaningful information
drop_cols.append("realTimePosition")
drop_cols.append("worthChange")
drop_cols.append("Unnamed: 0")

drop_cols

['realTimeWorth',
 'realTimePosition',
 'worthChange',
 'realTimePosition',
 'worthChange',
 'Unnamed: 0']

In [178]:
b

,id,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
0,7468,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1
1,4605,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2
2,7368,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3
3,3808,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4
4,3213,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5
...,...,...,...,...,...,...,...
2203,7578,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134
2204,9196,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134
2205,1175,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856
2206,9466,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978


In [179]:
b.drop(drop_cols, axis=1, inplace=True)
b

,id,Source,worth
0,7468,Technology ==> Amazon,112.0 BUSD
1,4605,Technology ==> Microsoft,90.0 BUSD
2,7368,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD
3,3808,Fashion & Retail ==> LVMH,72.0 BUSD
4,3213,Technology ==> Facebook,71.0 BUSD
...,...,...,...
2203,7578,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD
2204,9196,Manufacturing ==> electrical equipment,1.0 BUSD
2205,1175,Manufacturing ==> electrical equipment,1.0 BUSD
2206,9466,Manufacturing ==> printed circuit boards,1.0 BUSD


In [172]:
'''
def source_split(source, position):
    my_list = source.split("==>")
    return my_list[position].strip()
'''

'\ndef source_split(source, position):\n    my_list = source.split("==>")\n    return my_list[position].strip()\n'

In [180]:
# b["Class"] = b["Source"].apply(source_split, position=0)
# b["Company"] = b["Source"].apply(source_split, position=1)
b["Class"] = b["Source"].str.split("==>").apply(lambda x: x[0]).apply(str.strip)
b["Company"] = b["Source"].str.split("==>").apply(lambda x: x[1]).apply(str.strip)
b

,id,Source,worth,Class,Company
0,7468,Technology ==> Amazon,112.0 BUSD,Technology,Amazon
1,4605,Technology ==> Microsoft,90.0 BUSD,Technology,Microsoft
2,7368,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,Finance and Investments,Berkshire Hathaway
3,3808,Fashion & Retail ==> LVMH,72.0 BUSD,Fashion & Retail,LVMH
4,3213,Technology ==> Facebook,71.0 BUSD,Technology,Facebook
...,...,...,...,...,...
2203,7578,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,Fashion & Retail,"fashion, entertainment"
2204,9196,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment
2205,1175,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment
2206,9466,Manufacturing ==> printed circuit boards,1.0 BUSD,Manufacturing,printed circuit boards


In [182]:
b["Num_Worth"] = b["worth"].str.replace(" BUSD", "")
b = b.astype({"Num_Worth": float})
b

,id,Source,worth,Class,Company,Num_Worth
0,7468,Technology ==> Amazon,112.0 BUSD,Technology,Amazon,112.0
1,4605,Technology ==> Microsoft,90.0 BUSD,Technology,Microsoft,90.0
2,7368,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,Finance and Investments,Berkshire Hathaway,84.0
3,3808,Fashion & Retail ==> LVMH,72.0 BUSD,Fashion & Retail,LVMH,72.0
4,3213,Technology ==> Facebook,71.0 BUSD,Technology,Facebook,71.0
...,...,...,...,...,...,...
2203,7578,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,Fashion & Retail,"fashion, entertainment",1.0
2204,9196,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment,1.0
2205,1175,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment,1.0
2206,9466,Manufacturing ==> printed circuit boards,1.0 BUSD,Manufacturing,printed circuit boards,1.0


In [184]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 6 columns):
id           2208 non-null int64
Source       2208 non-null object
worth        2208 non-null object
Class        2208 non-null object
Company      2208 non-null object
Num_Worth    2208 non-null float64
dtypes: float64(1), int64(1), object(4)
memory usage: 103.6+ KB


In [136]:
b

,id,Unnamed: 0,Source,worth,Class,Company,Num_Worth
0,7468,52,Technology ==> Amazon,112.0 BUSD,Technology,Amazon,112.0
1,4605,53,Technology ==> Microsoft,90.0 BUSD,Technology,Microsoft,90.0
2,7368,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,Finance and Investments,Berkshire Hathaway,84.0
3,3808,55,Fashion & Retail ==> LVMH,72.0 BUSD,Fashion & Retail,LVMH,72.0
4,3213,56,Technology ==> Facebook,71.0 BUSD,Technology,Facebook,71.0
...,...,...,...,...,...,...,...
2203,7578,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,Fashion & Retail,"fashion, entertainment",1.0
2204,9196,2256,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment,1.0
2205,1175,2257,Manufacturing ==> electrical equipment,1.0 BUSD,Manufacturing,electrical equipment,1.0
2206,9466,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,Manufacturing,printed circuit boards,1.0


In [185]:
b.drop(["Source", "worth"], axis=1, inplace=True)
b

,id,Class,Company,Num_Worth
0,7468,Technology,Amazon,112.0
1,4605,Technology,Microsoft,90.0
2,7368,Finance and Investments,Berkshire Hathaway,84.0
3,3808,Fashion & Retail,LVMH,72.0
4,3213,Technology,Facebook,71.0
...,...,...,...,...
2203,7578,Fashion & Retail,"fashion, entertainment",1.0
2204,9196,Manufacturing,electrical equipment,1.0
2205,1175,Manufacturing,electrical equipment,1.0
2206,9466,Manufacturing,printed circuit boards,1.0


In [186]:
b.rename(columns={"Num_Worth": "Worth", "Class": "Sector"}, inplace=True)
b

,id,Sector,Company,Worth
0,7468,Technology,Amazon,112.0
1,4605,Technology,Microsoft,90.0
2,7368,Finance and Investments,Berkshire Hathaway,84.0
3,3808,Fashion & Retail,LVMH,72.0
4,3213,Technology,Facebook,71.0
...,...,...,...,...
2203,7578,Fashion & Retail,"fashion, entertainment",1.0
2204,9196,Manufacturing,electrical equipment,1.0
2205,1175,Manufacturing,electrical equipment,1.0
2206,9466,Manufacturing,printed circuit boards,1.0


In [259]:
b.sort_values(by="Worth", ascending=False, inplace=True)

In [263]:
b["Position"] = b.index + 1
b

,id,Sector,Company,Worth,Position
0,7468,Technology,Amazon,112.0,1
1,4605,Technology,Microsoft,90.0,2
2,7368,Finance and Investments,Berkshire Hathaway,84.0,3
3,3808,Fashion & Retail,LVMH,72.0,4
4,3213,Technology,Facebook,71.0,5
...,...,...,...,...,...
2149,4412,Manufacturing,auto components,1.0,2150
2148,1655,Gambling & Casinos,pachinko machines,1.0,2149
2147,8413,Technology,printed circuit boards,1.0,2148
2146,7136,Manufacturing,packaging,1.0,2147


In [268]:
b.to_csv("../data/processed/business_info.csv", index=False)

In [270]:
r

,id,name,position,Unnamed: 0
0,3705,ananT ASAVABHOKHIN & FAMILY,NaN,0
1,9321,ARASH FERDowsi,NaN,1
2,4524,banthOON LAMSAM & family,NaN,2
3,4810,biLL ADDERLEY & FAmily,NaN,3
4,2336,boris ZINGAREVIch,NaN,4
...,...,...,...,...
2255,7578,zhao XIAOQIANg,2204.0,2255
2256,9196,zhou liANGZHang,2205.0,2256
2257,1175,zhu xiNGming,2206.0,2257
2258,9466,zHUO JUn,2207.0,2258


In [273]:
r.drop(columns=["position", "Unnamed: 0"], axis=1, inplace=True)

In [310]:
r["name"] = r["name"].str.upper()
r

,id,name
0,3705,ANANT ASAVABHOKHIN & FAMILY
1,9321,ARASH FERDOWSI
2,4524,BANTHOON LAMSAM & FAMILY
3,4810,BILL ADDERLEY & FAMILY
4,2336,BORIS ZINGAREVICH
...,...,...
2255,7578,ZHAO XIAOQIANG
2256,9196,ZHOU LIANGZHANG
2257,1175,ZHU XINGMING
2258,9466,ZHUO JUN


In [448]:
p

,id,lastName,age,Unnamed: 0,gender,country,image
0,7468,bEZOS,54 years old,52,Male,USA,https://specials-images.forbesimg.com/imageser...
1,4605,gATEs,62 years old,53,Male,None,https://specials-images.forbesimg.com/imageser...
2,7368,bufFETt,87 years old,54,Male,None,https://specials-images.forbesimg.com/imageser...
3,3808,arnAULT,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...
4,3213,zuckERBERg,1985,56,Male,None,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...,...,...
2203,7578,zhAO,51 years old,2255,Male,China,https://specials-images.forbesimg.com/imageser...
2204,9196,zHOU,55 years old,2256,Male,None,https://specials-images.forbesimg.com/imageser...
2205,1175,zHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...
2206,9466,zhUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...


In [456]:
import re
def string2null(x):
    if x == "None":
        return None
    else:
        return x
    
def ToMayus(x):
    if x == None:
        return "azul"

In [457]:
p["newcol"] = p["country"].apply(string2null)
print(ToMayus("BEZOS"))

None


In [476]:
p

,id,lastName,age,Unnamed: 0,gender,country,image,newcol
0,7468,bEZOS,54 years old,52,Male,USA,https://specials-images.forbesimg.com/imageser...,USA
1,4605,gATEs,62 years old,53,Male,None,https://specials-images.forbesimg.com/imageser...,None
2,7368,bufFETt,87 years old,54,Male,None,https://specials-images.forbesimg.com/imageser...,None
3,3808,arnAULT,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,None
4,3213,zuckERBERg,1985,56,Male,None,https://specials-images.forbesimg.com/imageser...,None
...,...,...,...,...,...,...,...,...
2203,7578,zhAO,51 years old,2255,Male,China,https://specials-images.forbesimg.com/imageser...,China
2204,9196,zHOU,55 years old,2256,Male,None,https://specials-images.forbesimg.com/imageser...,None
2205,1175,zHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,China
2206,9466,zhUO,52 years old,2258,Female,None,https://specials-images.forbesimg.com/imageser...,None


In [481]:
ageset = set(p["age"])
ageset

def anito(x):
    if x == None:
        return None
    elif len(x) == 4:
        return str(2018 - int(x))
    else:
        return x[0:2]


In [483]:
p["newage"] = p["age"].apply(anito)
p

,id,lastName,age,Unnamed: 0,gender,country,image,newcol,newage
0,7468,bEZOS,54 years old,52,Male,USA,https://specials-images.forbesimg.com/imageser...,USA,54
1,4605,gATEs,62 years old,53,Male,None,https://specials-images.forbesimg.com/imageser...,None,62
2,7368,bufFETt,87 years old,54,Male,None,https://specials-images.forbesimg.com/imageser...,None,87
3,3808,arnAULT,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,None,69
4,3213,zuckERBERg,1985,56,Male,None,https://specials-images.forbesimg.com/imageser...,None,33
...,...,...,...,...,...,...,...,...,...
2203,7578,zhAO,51 years old,2255,Male,China,https://specials-images.forbesimg.com/imageser...,China,51
2204,9196,zHOU,55 years old,2256,Male,None,https://specials-images.forbesimg.com/imageser...,None,55
2205,1175,zHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,China,51
2206,9466,zhUO,52 years old,2258,Female,None,https://specials-images.forbesimg.com/imageser...,None,52


In [494]:
p.drop(columns=["lastName", "age", "Unnamed: 0", "country"], axis=1, inplace=True)
p

,id,gender,image,newcol,newage
0,7468,Male,https://specials-images.forbesimg.com/imageser...,USA,54
1,4605,Male,https://specials-images.forbesimg.com/imageser...,None,62
2,7368,Male,https://specials-images.forbesimg.com/imageser...,None,87
3,3808,None,https://specials-images.forbesimg.com/imageser...,None,69
4,3213,Male,https://specials-images.forbesimg.com/imageser...,None,33
...,...,...,...,...,...
2203,7578,Male,https://specials-images.forbesimg.com/imageser...,China,51
2204,9196,Male,https://specials-images.forbesimg.com/imageser...,None,55
2205,1175,None,https://specials-images.forbesimg.com/imageser...,China,51
2206,9466,Female,https://specials-images.forbesimg.com/imageser...,None,52


In [498]:
p = p.rename(columns={"newage": "age", "newcol": "country"})
p

,id,gender,image,country,age
0,7468,Male,https://specials-images.forbesimg.com/imageser...,USA,54
1,4605,Male,https://specials-images.forbesimg.com/imageser...,None,62
2,7368,Male,https://specials-images.forbesimg.com/imageser...,None,87
3,3808,None,https://specials-images.forbesimg.com/imageser...,None,69
4,3213,Male,https://specials-images.forbesimg.com/imageser...,None,33
...,...,...,...,...,...
2203,7578,Male,https://specials-images.forbesimg.com/imageser...,China,51
2204,9196,Male,https://specials-images.forbesimg.com/imageser...,None,55
2205,1175,None,https://specials-images.forbesimg.com/imageser...,China,51
2206,9466,Female,https://specials-images.forbesimg.com/imageser...,None,52


In [499]:
new_sex = {"F": "Female", "Female": "Female", "M": "Male", "Male": "Male", "None": None, None: None}
p["gender"] = p["gender"].map(new_sex)
p["gender"].value_counts(dropna=False)

Male      1486
NaN        550
Female     172
Name: gender, dtype: int64

In [500]:
p

,id,gender,image,country,age
0,7468,Male,https://specials-images.forbesimg.com/imageser...,USA,54
1,4605,Male,https://specials-images.forbesimg.com/imageser...,None,62
2,7368,Male,https://specials-images.forbesimg.com/imageser...,None,87
3,3808,None,https://specials-images.forbesimg.com/imageser...,None,69
4,3213,Male,https://specials-images.forbesimg.com/imageser...,None,33
...,...,...,...,...,...
2203,7578,Male,https://specials-images.forbesimg.com/imageser...,China,51
2204,9196,Male,https://specials-images.forbesimg.com/imageser...,None,55
2205,1175,None,https://specials-images.forbesimg.com/imageser...,China,51
2206,9466,Female,https://specials-images.forbesimg.com/imageser...,None,52


In [501]:
p.columns

Index(['id', 'gender', 'image', 'country', 'age'], dtype='object')

In [503]:
p = p[['id', 'gender', 'country', 'age', 'image']]

In [504]:
p

,id,gender,country,age,image
0,7468,Male,USA,54,https://specials-images.forbesimg.com/imageser...
1,4605,Male,None,62,https://specials-images.forbesimg.com/imageser...
2,7368,Male,None,87,https://specials-images.forbesimg.com/imageser...
3,3808,None,None,69,https://specials-images.forbesimg.com/imageser...
4,3213,Male,None,33,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...
2203,7578,Male,China,51,https://specials-images.forbesimg.com/imageser...
2204,9196,Male,None,55,https://specials-images.forbesimg.com/imageser...
2205,1175,None,China,51,https://specials-images.forbesimg.com/imageser...
2206,9466,Female,None,52,https://specials-images.forbesimg.com/imageser...


In [512]:
p

,id,gender,country,age,image
0,7468,Male,USA,54,https://specials-images.forbesimg.com/imageser...
1,4605,Male,None,62,https://specials-images.forbesimg.com/imageser...
2,7368,Male,None,87,https://specials-images.forbesimg.com/imageser...
3,3808,None,None,69,https://specials-images.forbesimg.com/imageser...
4,3213,Male,None,33,https://specials-images.forbesimg.com/imageser...
...,...,...,...,...,...
2203,7578,Male,China,51,https://specials-images.forbesimg.com/imageser...
2204,9196,Male,None,55,https://specials-images.forbesimg.com/imageser...
2205,1175,None,China,51,https://specials-images.forbesimg.com/imageser...
2206,9466,Female,None,52,https://specials-images.forbesimg.com/imageser...
